In [19]:
import requests
import re

url = 'https://es.wikipedia.org/wiki/National_Basketball_Association'
response = requests.get(url)

if response.status_code == 200:
    web_content = response.text
    
    # Eliminar las etiquetas HTML
    clean_text = re.sub(r'<[^>]*>', '', web_content)
    
    # Limpiar espacios en blanco adicionales
    clean_text = re.sub(r'\s+', ' ', clean_text)
    
    print("Texto de la página cargado:")
    print(clean_text[:1000])  # Muestra los primeros 1000 caracteres del texto limpio
else:
    raise Exception(f"Error al cargar la página: {response.status_code}")

Texto de la página cargado:
 National Basketball Association - Wikipedia, la enciclopedia libre (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available";var cookie=document.cookie.match(/(?:^|; )eswikimwclientpreferences=([^;]+)/);if(cookie){cookie[1].split('%2C').forEach(function(pref){className=className.replace(new RegExp('(^| )'+pref.replace(/-clientpref-\w+$|[^\w-]+/g,'')+'-clientpref-\\w+( |$)'),'$1'+pref+'$2');});}document.documentElement.className=className;}());RLCONF={"wgBreakFrames":false,

In [20]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [22]:
vector_store = Chroma(
    collection_name="rag1",
    embedding_function=embeddings,
    persist_directory="./datasets",
)

In [23]:
retriever = vector_store.as_retriever()

In [25]:
# Plantilla de conversación
conversation_template = """Please answer the following question based only on the given context:
{context}

Question: {question}
"""

# Crear el prompt a partir de la plantilla
prompt_template = ChatPromptTemplate.from_template(conversation_template)

# LLM local
local_llm = "tinyllama"
local_model = ChatOllama(model=local_llm)

# Cadena de procesamiento
qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | local_model
    | StrOutputParser()
)

In [27]:
qa_chain.invoke("What is National Basketball Association?")

'National Basketball Association (NBA) is a professional indoor basketball league in North America founded in 2016. The league consists of teams from Canada, the United States, and Mexico, with games played in major cities across the country. NBA has partnered with top American universities to offer scholarship opportunities for young players who wish to pursue a career in professional basketball.'